# Converting numeric data to sequence data: Example based on the Gapminder Life Expectancy data

*Author: Yuqi Liang*

*Date: 13 Feb 2026*

In this tutorial, we will explore the Gapminder Life Expectancy data and demonstrate how to convert numeric data into sequence data using **global deciles**.

**Data Source:** The data used in this tutorial is sourced from [Gapminder](https://www.gapminder.org/data/). Gapminder provides comprehensive datasets on various global indicators, including life expectancy at birth.

**Data Analysis and Cleaning:** We will transform the numeric dataset into a sequence dataset by:

* Loading the dataset.
* Converting the dataset to long format and handling missing values.
* Computing **global decile** thresholds (dividing all values into 10 equal-sized groups).
* Converting the dataset to wide format with sequence states.

Let's get started!

In [1]:
# Import necessary packages
import pandas as pd

## Life Expectancy

The data source for each country's life expectancy at birth can be found [here](https://www.gapminder.org/data/). This dataset contains life expectancy values for countries from 1800 to 2100.

In [2]:
# Load the dataset (wide format: countries as rows, years as columns)
file_path = "../data_sources/life_expectancy.csv"
df_wide = pd.read_csv(file_path)

# Convert from wide format to long format
# Melt the dataframe: keep 'name' and 'geo' as identifiers, convert year columns to rows
# Note: 'geo' is the country code column, we include it in id_vars so it doesn't get melted into 'year'
df_long = df_wide.melt(
    id_vars=['name', 'geo'],
    var_name='year',
    value_name='life_expectancy'
)

# Convert year to integer and life_expectancy to numeric
df_long['year'] = df_long['year'].astype(int)
df_long['life_expectancy'] = pd.to_numeric(df_long['life_expectancy'], errors='coerce')

# Rename 'name' to 'country' for consistency
df_long = df_long.rename(columns={'name': 'country'})

# Reorder columns (drop 'geo' as we don't need it for the analysis)
df_long = df_long[['country', 'year', 'life_expectancy']]

df_long

,country,year,life_expectancy
0,Afghanistan,1800,28.21
1,Angola,1800,26.98
2,Albania,1800,35.40
3,Andorra,1800,NaN
4,UAE,1800,30.70
...,...,...,...
58389,Samoa,2100,82.27
58390,Yemen,2100,82.08
58391,South Africa,2100,75.24
58392,Zambia,2100,74.48


In [3]:
# Missing values in the original dataset (before dropping)
n_total = len(df_long)
n_missing_life_expectancy = df_long['life_expectancy'].isna().sum()
n_missing_year = df_long['year'].isna().sum()
n_missing_country = df_long['country'].isna().sum()

print("Missing values in the original dataset:")
print(f"  Total rows: {n_total:,}")
print(f"  Missing in life_expectancy: {n_missing_life_expectancy:,} ({100 * n_missing_life_expectancy / n_total:.2f}%)")
print(f"  Missing in year: {n_missing_year:,}")
print(f"  Missing in country: {n_missing_country:,}")

# Drop rows with missing life_expectancy for downstream analysis
df_long = df_long.dropna(subset=['life_expectancy'])
print(f"\nAfter dropping rows with missing life_expectancy: {len(df_long):,} rows remain.")

Missing values in the original dataset:
  Total rows: 58,394
  Missing in life_expectancy: 1,581 (2.71%)
  Missing in year: 0
  Missing in country: 0

After dropping rows with missing life_expectancy: 56,813 rows remain.


## Global deciles

**Global deciles** divide all life expectancy values (across all years and countries) into 10 equal-sized groups. One set of thresholds is computed for the entire dataset. D1 = lowest 10%, D10 = highest 10%.

In [4]:
# Ensure life_expectancy is numeric and drop any remaining non-numeric values
# Filter to only numeric values before computing deciles
df_long_numeric = df_long[pd.to_numeric(df_long['life_expectancy'], errors='coerce').notna()].copy()
df_long_numeric['life_expectancy'] = pd.to_numeric(df_long_numeric['life_expectancy'], errors='coerce')

# Drop any rows that still have NaN after conversion (shouldn't happen, but just in case)
df_long_numeric = df_long_numeric.dropna(subset=['life_expectancy'])

# Verify the column is numeric
print(f"Data type of life_expectancy: {df_long_numeric['life_expectancy'].dtype}")
print(f"Number of rows for decile computation: {len(df_long_numeric):,}")

# Compute global deciles
df_long_numeric['decile_global'] = pd.qcut(
    df_long_numeric['life_expectancy'],
    q=10,
    labels=[
        'D1 (Very Low)', 'D2', 'D3', 'D4', 'D5',
        'D6', 'D7', 'D8', 'D9', 'D10 (Very High)'
    ]
)

# Convert to wide format: rows = country, columns = year, values = decile
df_global_deciles = df_long_numeric.pivot(index='country', columns='year', values='decile_global')

# Reset index and clean column names
df_global_deciles = df_global_deciles.reset_index()
df_global_deciles.columns.name = None

df_global_deciles

Data type of life_expectancy: float64
Number of rows for decile computation: 56,813


,country,1800,1801,1802,1803,1804,1805,1806,1807,1808,...,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100
0,Afghanistan,D1 (Very Low),D1 (Very Low),D1 (Very Low),D1 (Very Low),D1 (Very Low),D1 (Very Low),D1 (Very Low),D1 (Very Low),D1 (Very Low),...,D8,D8,D8,D8,D8,D9,D9,D9,D9,D9
1,Albania,D4,D4,D4,D4,D4,D4,D4,D4,D4,...,D10 (Very High),D10 (Very High),D10 (Very High),D10 (Very High),D10 (Very High),D10 (Very High),D10 (Very High),D10 (Very High),D10 (Very High),D10 (Very High)
2,Algeria,D1 (Very Low),D1 (Very Low),D1 (Very Low),D1 (Very Low),D1 (Very Low),D1 (Very Low),D1 (Very Low),D1 (Very Low),D1 (Very Low),...,D10 (Very High),D10 (Very High),D10 (Very High),D10 (Very High),D10 (Very High),D10 (Very High),D10 (Very High),D10 (Very High),D10 (Very High),D10 (Very High)
3,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,D10 (Very High),D10 (Very High),D10 (Very High),D10 (Very High),D10 (Very High),D10 (Very High),D10 (Very High),D10 (Very High),D10 (Very High),D10 (Very High)
4,Angola,D1 (Very Low),D1 (Very Low),D1 (Very Low),D1 (Very Low),D1 (Very Low),D1 (Very Low),D1 (Very Low),D1 (Very Low),D1 (Very Low),...,D8,D8,D8,D8,D8,D8,D8,D9,D9,D9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,Venezuela,D3,D3,D3,D3,D3,D3,D3,D3,D3,...,D10 (Very High),D10 (Very High),D10 (Very High),D10 (Very High),D10 (Very High),D10 (Very High),D10 (Very High),D10 (Very High),D10 (Very High),D10 (Very High)
190,Vietnam,D2,D2,D2,D2,D2,D2,D2,D2,D2,...,D10 (Very High),D10 (Very High),D10 (Very High),D10 (Very High),D10 (Very High),D10 (Very High),D10 (Very High),D10 (Very High),D10 (Very High),D10 (Very High)
191,Yemen,D1 (Very Low),D1 (Very Low),D1 (Very Low),D1 (Very Low),D1 (Very Low),D1 (Very Low),D1 (Very Low),D1 (Very Low),D1 (Very Low),...,D9,D9,D9,D9,D9,D9,D9,D9,D9,D9
192,Zambia,D3,D3,D3,D3,D3,D3,D3,D3,D3,...,D8,D8,D8,D8,D8,D8,D8,D8,D8,D8


In [5]:
# Save to CSV
df_global_deciles.to_csv('country_life_expectancy_global_deciles.csv', index=False)

## Missing values in the final output

The final wide-format data (`df_global_deciles`) may contain missing values (NaN) where a country has no life expectancy record for a given year. Below we report where they are and how many.

In [6]:
# Count missing values in the final output
year_cols = [c for c in df_global_deciles.columns if c != 'country']
n_total = df_global_deciles.shape[0] * len(year_cols)
n_missing = df_global_deciles[year_cols].isna().sum().sum()

print("Missing values in df_global_deciles (final output):")
print(f"  Total cells (countries × years): {n_total:,}")
print(f"  Missing cells: {n_missing:,} ({100 * n_missing / n_total:.2f}%)")

# Where are the missing values? By country (how many years missing per country)
missing_by_country = df_global_deciles[year_cols].isna().sum(axis=1)
countries_with_missing = missing_by_country[missing_by_country > 0]
if len(countries_with_missing) > 0:
    print(f"\nCountries with missing values ({len(countries_with_missing)} countries):")
    for idx in countries_with_missing.index[:10]:  # show first 10
        c = df_global_deciles.loc[idx, 'country']
        print(f"  - {c}: {countries_with_missing[idx]:,} years missing")
    if len(countries_with_missing) > 10:
        print(f"  ... and {len(countries_with_missing) - 10} more countries")

# By year (how many countries missing per year)
missing_by_year = df_global_deciles[year_cols].isna().sum()
years_with_missing = missing_by_year[missing_by_year > 0]
if len(years_with_missing) > 0:
    print(f"\nYears with missing values ({len(years_with_missing)} years):")
    for y in list(years_with_missing.index)[:10]:
        print(f"  - {y}: {years_with_missing[y]:,} countries missing")
    if len(years_with_missing) > 10:
        print(f"  ... and {len(years_with_missing) - 10} more years")
else:
    print("\nNo missing values in the final output.")

Missing values in df_global_deciles (final output):
  Total cells (countries × years): 58,394
  Missing cells: 1,581 (2.71%)

Countries with missing values (10 countries):
  - Andorra: 150 years missing
  - Dominica: 150 years missing
  - Liechtenstein: 231 years missing
  - Marshall Islands: 150 years missing
  - Monaco: 150 years missing
  - Nauru: 150 years missing
  - Palau: 150 years missing
  - San Marino: 150 years missing
  - St. Kitts and Nevis: 150 years missing
  - Tuvalu: 150 years missing

Years with missing values (231 years):
  - 1800: 10 countries missing
  - 1801: 10 countries missing
  - 1802: 10 countries missing
  - 1803: 10 countries missing
  - 1804: 10 countries missing
  - 1805: 10 countries missing
  - 1806: 10 countries missing
  - 1807: 10 countries missing
  - 1808: 10 countries missing
  - 1809: 10 countries missing
  ... and 221 more years
